In [1]:
import inro.modeller as m
import os
import glob
mm = m.Modeller()
eb = mm.emmebank
import inro.emme.matrix as _matrix
import pandas as pd
import numpy as np
from copy import deepcopy

matrix_calculator = m.Modeller().tool("inro.emme.matrix_calculation.matrix_calculator")
create_matrix =  m.Modeller().tool("inro.emme.data.matrix.create_matrix")
change_matrix = m.Modeller().tool("inro.emme.data.matrix.change_matrix_properties") 
import_matrices = mm.tool("tmg.input_output.import_binary_matrix")  
matrix_transaction = mm.tool('inro.emme.data.matrix.matrix_transaction')
create_extra = m.Modeller().tool("inro.emme.data.extra_attribute.create_extra_attribute")
network_calc= m.Modeller().tool("inro.emme.network_calculation.network_calculator")
process_partition = m.Modeller().tool("inro.emme.data.zone_partition.partition_transaction")

def clear_matrices(emmebank, exception_array = [""], type = "FULL"):
    for matrix in emmebank.matrices():
        if matrix.type == type:
            if matrix.id not in exception_array:
                emmebank.delete_matrix(matrix.id)

def import_matrices_from_directory(matrix_folder, matrix_list, extension, scenario):
    #imports all matrices in directory and subfolders of the specified extension type
    #matrix_list should be in the format:
    #   {file_name1: matrix_num1, file_name2: matrix_num2, ...}
    import os

    for root, dirs, files in os.walk(matrix_folder):
        for matrix_file in files:
            if matrix_file.endswith(extension):
                name = os.path.splitext(matrix_file)[0]
                if name in matrix_list:
                    matrix_number = int(matrix_list[name][2:])
                    import_matrices(4, matrix_number ,os.path.join(root, matrix_file),scenario, name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_") )
                    print "Imported matrix " + matrix_list[name]
                    change_matrix(matrix = matrix_list[name],
                        matrix_name = name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_")[:40],
                        matrix_description = name)
    
    #check
    for name in matrix_list:
        if eb.matrix(matrix_list[name]): 
            n_matrix = eb.matrix(matrix_list[name]).get_numpy_data(scenario)
            if n_matrix.sum() == 0 :
                print "Matrix %s was not imported, or contains no values!" % name
        else:
            print "Matrix %s was not imported!" % name
    print "Finished Importing Matrices"
    
#obtain the minimum transit travel time between all modes.
#uses outputs from HDR transit time toolbox
mcSpec = {
    "expression": None,
    "result": "",
    "constraint": {
        "by_value": None,
        "by_zone": None
    },
    "aggregation": {
        "origins": None,
        "destinations": None
    },
    "type": "MATRIX_CALCULATION"
}


def get_mc_spec(result, expression, agg_o, agg_d, con_o, con_d, con_val):
    specification = {
        "type": "MATRIX_CALCULATION",
        "result": result,
        "expression":  expression,
        "aggregation": {
            "origins": agg_o,
            "destinations": agg_d
        },
        "constraint": {
            "by_value": con_val,
            "by_zone": {
                "origins": con_o,
                "destinations": con_d
            }
        }
    }
    return specification

In [2]:
#scenario folder names

#BAU
sc_2051_BAU_ogs = "2051\\Short List\\BAU\\BAU_ShortList_APTG_2051_GGH_MTP_V106A3_CS222_20201127"

#Network 1
sc_2051_n1_aptg = "2051\\Short List\\Network 1\\N1_ShortList_APTG_2051_GGH_MTP_V106A3_CS222_20201217"
sc_2051_n1_luv1 = "2051\\Short List\\Network 1\\N1_SList_LUV1_2051_GGH_MTP_V106A3_CS222_20210108"
sc_2051_n1_luv2 = "2051\\Short List\\Network 1\\N1_SList_LUV2_2051_GGH_MTP_V106A3_CS222_20210204"
sc_2051_n1_luv3 = "2051\\Short List\\Network 1\\N1_SList_LUV3_2051_GGH_MTP_V106A3_CS222_20201230"

#Network 2
sc_2051_n2_aptg = "2051\\Short List\\Network 2\\N2_ShortList_APTG_2051_GGH_MTP_V106A3_CS222_20201217"
sc_2051_n2_luv1 = "2051\\Short List\\Network 2\\N2_SList_LUV1_2051_GGH_MTP_V106A3_CS222_20210108"
sc_2051_n2_luv2 = "2051\\Short List\\Network 2\\N2_SList_LUV2_2051_GGH_MTP_V106A3_CS222_20210204"
sc_2051_n2_luv3 = "2051\\Short List\\Network 2\\N2_SList_LUV3_2051_GGH_MTP_V106A3_CS222_20201230"

#Transit Sens\
sc_2051_transimpro1_aptg = "2051\\Short List\\Policy Runs\\N2_SList_APTG_TransitImpro1_2051_GGH_MTP_V106A3_HS022_20210209"
sc_2051_transimpro2_aptg = "2051\\Short List\\Policy Runs\\N2_SList_APTG_TransitImpro2_2051_GGH_MTP_V106A3_HS022_20210209"
sc_2051_bau_localtrans_aptg = "2051\\Short List\\Policy Runs\\BAU_Local-transit-sensitivity_SList_APTG_2051_GGH_MTP_V106A3_HS022_20210322"
sc_2051_n2_localtrans_aptg = "2051\\Short List\\Policy Runs\\SNL2_Local-transit-sensitivity_SList_APTG_2051_GGH_MTP_V106A3_HS022_20210322"

#Preferred Network
sc_2051_prefN_aptg = "2051\\Preferred Network\\Preferred_Network_SList_APTG_2051_GGH_MTP_V106A3_CS222_20210305"

## zone grouping definitions ## 
zones_pd1 = "gf1"
zones_airports = "ga1"
zones_ugc = "gb4"
zones_inner = "gc1"
zones_outer = "gc2"
zones_university = "gd1"
zones_employment = "ge1"
zones_hospitals = "gh1"
shed_central = "gs10;gs11"
rural_central = "gs11"
shed_east = "gs20;gs21"
rural_east = "gs21"
shed_midwest = "gs30;gs31"
rural_midwest = "gs31"
shed_southwest = "gs40;gs41"
rural_southwest = "gs41"
shed_west = "gs50;gs51"
rural_west = "gs51"
shed_north = "gs60;gs61"
rural_north = "gs61"
shed_niagara = "gs70;gs71"
rural_niagara = "gs71"
shed_urban = "gs10;gs20;gs30;gs40;gs50;gs60;gs70"
shed_rural = "gs11;gs21;gs31;gs41;gs51;gs61;gs71"
urban_lu = "gb2;gb3;gb4"

# Zone groupings with stations
zones_pd1_st = "gg1"
shed_central_st = "gu10;gu11"
shed_east_st = "gu20;gu21"
shed_midwest_st = "gu30;gu31"
shed_southwest_st = "gu40;gu41"
shed_west_st = "gu50;gu51"
shed_north_st = "gu60;gu61"
shed_niagara_st = "gu70;gu71"

#auto occupancy
sov_oc = 1.2
hov2_oc = 2.2
hov3_oc = 3.3

subway_fare = 1.98
go_base_fare = 4.07
go_dist_fare = 0.0825
parking_factor = 0.4 #Daily to AM

#number of hours for transit assignment
num_hrs_am = 2
#num_hrs_md = 6

In [3]:
pop_o_matrix_ogs = "mo19"
emp_o_matrix_ogs = "mo20"
senior_o_matrix_ogs = "mo21"
low_inc_o_matrix_ogs = "mo22"# number of low inc households
youth_o_matrix_ogs = "mo23"
pop_d_matrix_ogs = "md92"
emp_d_matrix_ogs = "md93"
low_income_zones_ogs = "mo24"

low_income_zones_stations_ogs_bau = "mo43"
low_income_zones_stations_ogs_n1 = "mo44"
low_income_zones_stations_ogs_n2 = "mo45"
low_income_zones_stations_ogs_n2 = "mo45"

pop_o_matrix_luv1 = "mo25"
emp_o_matrix_luv1 = "mo26"
senior_o_matrix_luv1 = "mo27"
low_inc_o_matrix_luv1 = "mo28"# number of low inc households
youth_o_matrix_luv1 = "mo29"
pop_d_matrix_luv1 = "md94"
emp_d_matrix_luv1 = "md95"
low_income_zones_luv1 = "mo30"


pop_o_matrix_luv2 = "mo31"
emp_o_matrix_luv2 = "mo32"
senior_o_matrix_luv2 = "mo33"
low_inc_o_matrix_luv2 = "mo34"# number of low inc households
youth_o_matrix_luv2 = "mo35"
pop_d_matrix_luv2 = "md96"
emp_d_matrix_luv2 = "md97"
low_income_zones_luv2 = "mo36"

pop_o_matrix_luv3 = "mo37"
emp_o_matrix_luv3 = "mo38"
senior_o_matrix_luv3 = "mo39"
low_inc_o_matrix_luv3 = "mo40"# number of low inc households
youth_o_matrix_luv3 = "mo41"
pop_d_matrix_luv3 = "md98"
emp_d_matrix_luv3 = "md99"
low_income_zones_luv3 = "mo42"

low_income_zones_stations_luv1_n1 = "mo46"
low_income_zones_stations_luv2_n1 = "mo47"                       
low_income_zones_stations_luv3_n1 = "mo48"
low_income_zones_stations_luv1_n2 = "mo49"
low_income_zones_stations_luv2_n2 = "mo50"
low_income_zones_stations_luv3_n2 = "mo51"

hospital_d = "md90"
postsec_d = "md91"
parking_d = "md24"

truck_folder = r"\\tore-infs01\PWExternal\451010\281829_GGH\GGHM Runs\2051\Long List\BAU\MTO_2051_GGH_MTP_5002_V106A3_HS032_20200117\Assignable matrices"
tt_dir = r"\\tore-infs01\Models\10039241 - GGH Multimodal Transportation\HDR_transit_time_results\20200212_2051_BAU_HS - Peak"
tt_dir_op = r"\\tore-infs01\Models\10039241 - GGH Multimodal Transportation\HDR_transit_time_results\20200212_2051_BAU_HS - Off-Peak"
hdr_bau_peak = "mf160"
hdr_bau_offpeak = "mf170"
local_fare_mat = "mf180"

In [4]:
#Matrix naming for inputs

# peak period - 7 hrs
sov_demand_pk = "TRIPS SOV peak"
hov2_demand_pk = "TRIPS HOV2 peak"
hov3_demand_pk = "TRIPS HOV3 peak"
bike_demand_pk = "TRIPS BIKE peak"
walk_demand_pk = "TRIPS WALK peak"
transit_demand_pk = "TRIPS TOTAL TRANSIT peak"

sov_demand_offpk = "TRIPS SOV offpeak"
hov2_demand_offpk = "TRIPS HOV2 offpeak"
hov3_demand_offpk = "TRIPS HOV3 offpeak"
bike_demand_offpk = "TRIPS BIKE offpeak"
walk_demand_offpk = "TRIPS WALK offpeak"
transit_demand_offpk = "TRIPS TOTAL TRANSIT offpeak"

#Truck matrices
truck_l_demand_am = "am_light_2051_20191227"
truck_m_demand_am = "am_medium_2051_20191227"
truck_h_demand_am = "am_heavy_2051_20191227"
truck_l_demand_md = "md_light_2051_20191227"
truck_m_demand_md = "md_medium_2051_20191227"
truck_h_demand_md = "md_heavy_2051_20191227"

#transit demand
trips_am_expressbus = "trips_peak_transit_express_bus"
trips_am_localbus ="trips_peak_transit_local_bus"
trips_am_rapidbus ="trips_peak_transit_rapid_bus"
trips_am_subway ="trips_peak_transit_subway"
trips_am_train ="trips_peak_transit_train"
trips_am_ztsa ="trips_peak_transit_ztsa"
trips_am_walk ="trips_peak_active_walk"
trips_am_bike ="trips_peak_active_bike"

trips_md_expressbus = "trips_offpeak_transit_express_bus"
trips_md_localbus ="trips_offpeak_transit_local_bus"
trips_md_rapidbus ="trips_offpeak_transit_rapid_bus"
trips_md_subway ="trips_offpeak_transit_subway"
trips_md_train ="trips_offpeak_transit_train"
trips_md_ztsa ="trips_offpeak_transit_ztsa"
trips_md_walk ="trips_offpeak_active_walk"
trips_md_bike ="trips_offpeak_active_bike"

trip_am_auto1nt = "trips_am_peak_auto1_nt"
trip_am_auto1t = "trips_am_peak_auto1_toll" 
trip_am_auto2nth = "trips_am_peak_auto2_nt_hov"
trip_am_auto2ntnh = "trips_am_peak_auto2_nt_nh" 
trip_am_auto2th = "trips_am_peak_auto2_toll_hov" 
trip_am_auto2tnh ="trips_am_peak_auto2_toll_nh" 
trip_am_auto3nth ="trips_am_peak_auto3_nt_hov" 
trip_am_auto3ntnh ="trips_am_peak_auto3_nt_nh" 
trip_am_auto3th ="trips_am_peak_auto3_toll_hov" 
trip_am_auto3tnh ="trips_am_peak_auto3_toll_nh" 
trip_md_auto1nt ="trips_midday_auto1_nt" 
trip_md_auto1t ="trips_midday_auto1_toll"
trip_md_auto2nth ="trips_midday_auto2_nt_hov" 
trip_md_auto2ntnh ="trips_midday_auto2_nt_nh" 
trip_md_auto2th ="trips_midday_auto2_toll_hov" 
trip_md_auto2tnh ="trips_midday_auto2_toll_nh" 
trip_md_auto3nth ="trips_midday_auto3_nt_hov" 
trip_md_auto3ntnh ="trips_midday_auto3_nt_nh"
trip_md_auto3th ="trips_midday_auto3_toll_hov" 
trip_md_auto3tnh ="trips_midday_auto3_toll_nh" 

# distance
sov_dist_am = "skim.am_peak.auto.sov.dist"
sovt_dist_am = "skim.am_peak.auto.sovt.dist"
sov_dist_md = "skim.midday.auto.sov.dist"
hov2t_dist_am = "skim.am_peak.auto.hov2t.dist"
hov2_dist_am = "skim.am_peak.auto.hov2.dist"
hov3t_dist_am = "skim.am_peak.auto.hov3t.dist"
hov3_dist_am = "skim.am_peak.auto.hov3.dist"

#costs
sov_toll_am = "skim.am_peak.auto.sovt.toll_cost"
hov2_toll_am = "skim.am_peak.auto.hov2t.toll_cost"
hov3_toll_am = "skim.am_peak.auto.hov3t.toll_cost"


pk_gobus_fare = "skim.peak.transit.go_bus.fares"
pk_subway_dist ="skim.peak.transit.subway.distance"
pk_train_dist ="skim.peak.transit.train.distance"
pk_ztsa_fare ="skim.peak.transit.ztsa.fares"

In [5]:
zones_array = ["all",zones_pd1,shed_central,shed_east,shed_midwest,shed_southwest,shed_west,shed_north,shed_niagara]
rural_array = [shed_rural,zones_pd1,rural_central,rural_east,rural_midwest,rural_southwest,rural_west,rural_north,rural_niagara]
zones_array_2 = [zones_pd1,shed_central,shed_east,shed_midwest,shed_southwest,shed_west,shed_north,shed_niagara]

zones_array_st = ["all",zones_pd1_st,shed_central_st,shed_east_st,shed_midwest_st,shed_southwest_st,shed_west_st,shed_north_st,shed_niagara_st]

In [6]:
# scens = {"2051_BAU":[sc_2051_BAU,[401,402,403],[pop_o_matrix_ogs,emp_o_matrix_ogs,senior_o_matrix_ogs,
#                                                 low_inc_o_matrix_ogs,youth_o_matrix_ogs,pop_d_matrix_ogs,
#                                                 emp_d_matrix_ogs,low_income_zones_ogs,
#                                                 low_income_zones_stations_ogs_bau]],
#         "2051_BAU_OGS":[sc_2051_BAU_ogs,[404,405,406],[pop_o_matrix_ogs,emp_o_matrix_ogs,senior_o_matrix_ogs,
#                                                 low_inc_o_matrix_ogs,youth_o_matrix_ogs,pop_d_matrix_ogs,
#                                                 emp_d_matrix_ogs,low_income_zones_ogs,
#                                                 low_income_zones_stations_ogs_bau]],
#          "2051_N1_APTG":[sc_2051_N1_aptg,[501,502,503],[pop_o_matrix_ogs,emp_o_matrix_ogs,senior_o_matrix_ogs,
#                                                 low_inc_o_matrix_ogs,youth_o_matrix_ogs,pop_d_matrix_ogs,
#                                                 emp_d_matrix_ogs,low_income_zones_ogs,
#                                                 low_income_zones_stations_ogs_n1]],
#          "2051_N1_LUV1":[sc_2051_N1_luv1,[504,505,506],[pop_o_matrix_luv1,emp_o_matrix_luv1,senior_o_matrix_luv1,
#                                                 low_inc_o_matrix_luv1,youth_o_matrix_luv1,pop_d_matrix_luv1,
#                                                 emp_d_matrix_luv1,low_income_zones_luv1,
#                                                 low_income_zones_stations_luv1_n1]],
#         "2051_N2_APTG":[sc_2051_N2_aptg,[601,602,603],[pop_o_matrix_ogs,emp_o_matrix_ogs,senior_o_matrix_ogs,
#                                                 low_inc_o_matrix_ogs,youth_o_matrix_ogs,pop_d_matrix_ogs,
#                                                 emp_d_matrix_ogs,low_income_zones_ogs,
#                                                 low_income_zones_stations_ogs_n2]],
#          "2051_N2_LUV1":[sc_2051_N2_luv1,[604,605,606],[pop_o_matrix_luv1,emp_o_matrix_luv1,senior_o_matrix_luv1,
#                                                 low_inc_o_matrix_luv1,youth_o_matrix_luv1,pop_d_matrix_luv1,
#                                                 emp_d_matrix_luv1,low_income_zones_luv1,
#                                                 low_income_zones_stations_luv1_n2]],
#         "2051_N3_APTG":[sc_2051_N3_aptg,[701,702,703],[pop_o_matrix_ogs,emp_o_matrix_ogs,senior_o_matrix_ogs,
#                                                 low_inc_o_matrix_ogs,youth_o_matrix_ogs,pop_d_matrix_ogs,
#                                                 emp_d_matrix_ogs,low_income_zones_ogs,
#                                                 low_income_zones_stations_ogs_n3]],
#         "2051_N3_LUV2":[sc_2051_N3_luv2,[704,705,706],[pop_o_matrix_luv2,emp_o_matrix_luv2,senior_o_matrix_luv2,
#                                                 low_inc_o_matrix_luv2,youth_o_matrix_luv2,pop_d_matrix_luv2,
#                                                 emp_d_matrix_luv2,low_income_zones_luv2,
#                                                 low_income_zones_stations_luv2_n3]],
#         "2051_N4_APTG":[sc_2051_N4_aptg,[801,802,803],[pop_o_matrix_ogs,emp_o_matrix_ogs,senior_o_matrix_ogs,
#                                                 low_inc_o_matrix_ogs,youth_o_matrix_ogs,pop_d_matrix_ogs,
#                                                 emp_d_matrix_ogs,low_income_zones_ogs,
#                                                 low_income_zones_stations_ogs_n4]],
#         "2051_N4_LUV3":[sc_2051_N4_luv3,[804,805,806],[pop_o_matrix_luv3,emp_o_matrix_luv3,senior_o_matrix_luv3,
#                                                 low_inc_o_matrix_luv3,youth_o_matrix_luv3,pop_d_matrix_luv3,
#                                                 emp_d_matrix_luv3,low_income_zones_luv3,
#                                                 low_income_zones_stations_luv3_n4]],
#         "2051_N5_APTG":[sc_2051_N5_aptg,[901,902,903],[pop_o_matrix_ogs,emp_o_matrix_ogs,senior_o_matrix_ogs,
#                                                 low_inc_o_matrix_ogs,youth_o_matrix_ogs,pop_d_matrix_ogs,
#                                                 emp_d_matrix_ogs,low_income_zones_ogs,
#                                                 low_income_zones_stations_ogs_n5]],
#         "2051_N5_LUV3":[sc_2051_N5_luv3,[904,905,906],[pop_o_matrix_luv3,emp_o_matrix_luv3,senior_o_matrix_luv3,
#                                                 low_inc_o_matrix_luv3,youth_o_matrix_luv3,pop_d_matrix_luv3,
#                                                 emp_d_matrix_luv3,low_income_zones_luv3,
#                                                 low_income_zones_stations_luv3_n5]]} #AM,MD,PM

In [7]:
# scens = {"2051_N1_LUV1":[sc_2051_n1_luv1,[504,505,506],[pop_o_matrix_luv1,emp_o_matrix_luv1,senior_o_matrix_luv1,
#                 low_inc_o_matrix_luv1,youth_o_matrix_luv1,pop_d_matrix_luv1,
#                 emp_d_matrix_luv1,low_income_zones_luv1,
#                 low_income_zones_stations_luv1_n1]],
# "2051_N1_LUV2":[sc_2051_n1_luv2,[507,508,509],[pop_o_matrix_luv2,emp_o_matrix_luv2,senior_o_matrix_luv2,
#                 low_inc_o_matrix_luv2,youth_o_matrix_luv2,pop_d_matrix_luv2,
#                 emp_d_matrix_luv2,low_income_zones_luv2,
#                 low_income_zones_stations_luv2_n1]],
# "2051_N1_LUV3":[sc_2051_n1_luv3,[510,511,512],[pop_o_matrix_luv3,emp_o_matrix_luv3,senior_o_matrix_luv3,
#                 low_inc_o_matrix_luv3,youth_o_matrix_luv3,pop_d_matrix_luv3,
#                 emp_d_matrix_luv3,low_income_zones_luv3,
#                 low_income_zones_stations_luv3_n1]],
# "2051_N2_LUV1":[sc_2051_n2_luv1,[604,605,606],[pop_o_matrix_luv1,emp_o_matrix_luv1,senior_o_matrix_luv1,
#                 low_inc_o_matrix_luv1,youth_o_matrix_luv1,pop_d_matrix_luv1,
#                 emp_d_matrix_luv1,low_income_zones_luv1,
#                 low_income_zones_stations_luv1_n2]],
# "2051_N2_LUV2":[sc_2051_n2_luv2,[607,608,609],[pop_o_matrix_luv2,emp_o_matrix_luv2,senior_o_matrix_luv2,
#                 low_inc_o_matrix_luv2,youth_o_matrix_luv2,pop_d_matrix_luv2,
#                 emp_d_matrix_luv2,low_income_zones_luv2,
#                 low_income_zones_stations_luv2_n2]],
# "2051_N2_LUV3":[sc_2051_n2_luv3,[610,611,612],[pop_o_matrix_luv3,emp_o_matrix_luv3,senior_o_matrix_luv3,
#                 low_inc_o_matrix_luv3,youth_o_matrix_luv3,pop_d_matrix_luv3,
#                emp_d_matrix_luv3,low_income_zones_luv3,
# #                 low_income_zones_stations_luv3_n2]]}
# scens = {"2051_Pref_aptg":[sc_2051_prefN_aptg,[701,702,703],[pop_o_matrix_ogs,emp_o_matrix_ogs,senior_o_matrix_ogs,
#                     low_inc_o_matrix_ogs,youth_o_matrix_ogs,pop_d_matrix_ogs,
#                     emp_d_matrix_ogs,low_income_zones_ogs,
# #                     low_income_zones_stations_ogs_n2]]}
# scens = {"2051_N2_localtranssens":[sc_2051_n2_localtrans_aptg,[639,640,641],[pop_o_matrix_ogs,emp_o_matrix_ogs,senior_o_matrix_ogs,
#                                  low_inc_o_matrix_ogs,youth_o_matrix_ogs,pop_d_matrix_ogs,
#                                  emp_d_matrix_ogs,low_income_zones_ogs,
#                                  low_income_zones_stations_ogs_n2]],
#         "2051_BAU_localtranssens":[sc_2051_bau_localtrans_aptg,[413,414,415],[pop_o_matrix_ogs,emp_o_matrix_ogs,senior_o_matrix_ogs,
#                                     low_inc_o_matrix_ogs,youth_o_matrix_ogs,pop_d_matrix_ogs,
#                                     emp_d_matrix_ogs,low_income_zones_ogs,
#                                     low_income_zones_stations_ogs_n2]]}
# scens = {"2051_Pref_aptg":[sc_2051_prefN_aptg,[701,702,703],[pop_o_matrix_ogs,emp_o_matrix_ogs,senior_o_matrix_ogs,
#                     low_inc_o_matrix_ogs,youth_o_matrix_ogs,pop_d_matrix_ogs,
#                     emp_d_matrix_ogs,low_income_zones_ogs,
# #                     low_income_zones_stations_ogs_n2]]}
# scens = {"2051_BAU_OGS":[sc_2051_BAU_ogs,[404,405,406],[pop_o_matrix_ogs,emp_o_matrix_ogs,senior_o_matrix_ogs,
#                                                 low_inc_o_matrix_ogs,youth_o_matrix_ogs,pop_d_matrix_ogs,
#                                                 emp_d_matrix_ogs,low_income_zones_ogs,
#                                                 low_income_zones_stations_ogs_bau]]}

In [8]:
scens = {"2051_N2_Impro1":[sc_2051_transimpro1_aptg,[631,632,633],[pop_o_matrix_ogs,emp_o_matrix_ogs,senior_o_matrix_ogs,
                                                 low_inc_o_matrix_ogs,youth_o_matrix_ogs,pop_d_matrix_ogs,
                                                 emp_d_matrix_ogs,low_income_zones_ogs,
                                                 low_income_zones_stations_ogs_n2]],
         "2051_N2_Impro2":[sc_2051_transimpro2_aptg,[634,635,636],[pop_o_matrix_ogs,emp_o_matrix_ogs,senior_o_matrix_ogs,
                                                low_inc_o_matrix_ogs,youth_o_matrix_ogs,pop_d_matrix_ogs,
                                                emp_d_matrix_ogs,low_income_zones_ogs,
                                                low_income_zones_stations_ogs_n2]]}

In [9]:
all_auto = [trip_am_auto1nt, trip_am_auto1t, trip_am_auto2nth, trip_am_auto2ntnh,trip_am_auto2th,
            trip_am_auto2tnh,trip_am_auto3nth,trip_am_auto3ntnh,trip_am_auto3th,trip_am_auto3tnh]
auto_sov_dist_modes = [trip_am_auto1nt, trip_am_auto2ntnh,trip_am_auto3ntnh]
auto_sovt_dist_modes = [trip_am_auto1t, trip_am_auto2tnh,trip_am_auto3tnh]
toll_hov2 = [trip_am_auto2th,trip_am_auto2tnh]
toll_hov3 = [trip_am_auto3th,trip_am_auto3tnh]

op_cost = 0.1428
subway_phf = 0.59
go_rail_phf = 0.75
local_bus_phf = 0.64
go_bus_phf = 0.57
rapid_bus_phf = 0.55

partition_pth = r"\\tore-infs01\Models\10039241 - GGH Multimodal Transportation\EMME Databanks\GGH_Multimodal\Database - Short List Runs"

In [14]:
out_file = r"C:\Users\PECHEN\Documents\20210329AvgTripCost_TransImpro.csv"
moef = open(out_file, 'w')
moef.write("Scen,MOE,MOE_ID," + ",".join(zones_array) + "\n")
for scen in scens.keys():
    print("Now processing {}".format(scen))
    test_sc_num = scens[scen][1][0]
    test_sc = eb.scenario(test_sc_num)
    ntwk = test_sc.get_network()
    scenario_folder = scens[scen][0]
    matrix_folder = r"\\tore-infs01\PWExternal\451010\281829_GGH\GGHM Runs" + "\\" + scenario_folder
    
    pop_o_matrix = scens[scen][2][0]
    low_income_zones_stations = scens[scen][2][8]
    
    if scen[5] == "B":
        name = "2051_BAU"
    else:
        name = scen[:7]
    partition_fle = "ensemble_addStations_" + name + ".301"
    process_partition(transaction_file = os.path.join(partition_pth,partition_fle),
                      scenario = test_sc)
    print("partition updated")
    
    clear_matrices(eb,[hdr_bau_peak, hdr_bau_offpeak,local_fare_mat],"FULL")

    matrix_list = {pk_gobus_fare:"",
    pk_subway_dist:"",
    pk_train_dist:"",
    pk_ztsa_fare:"",
    trips_am_expressbus:"",
    trips_am_localbus:"",
    trips_am_rapidbus:"",
    trips_am_subway:"",
    trips_am_train:"",
    trips_am_ztsa:"",
    trip_am_auto1nt:"",
    trip_am_auto1t:"",
    trip_am_auto2nth:"",
    trip_am_auto2ntnh:"",
    trip_am_auto2th:"",
    trip_am_auto2tnh:"",
    trip_am_auto3nth:"",
    trip_am_auto3ntnh:"",
    trip_am_auto3th:"",
    trip_am_auto3tnh:"",
    sov_toll_am:"",
    hov2_toll_am:"",
    hov3_toll_am:"",
    sov_dist_am:"",
    hov2t_dist_am:"",
    hov2_dist_am:"",
    hov3t_dist_am:"",
    hov3_dist_am:"",
    sovt_dist_am:""}

    for matrix_name in matrix_list.keys():
        matrix_list[matrix_name] = eb.available_matrix_identifier('FULL')
        print matrix_name
        create_matrix(matrix_id = matrix_list[matrix_name],
                      matrix_name= matrix_name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_").strip()[:40],
                      matrix_description = matrix_name)

    import_matrices_from_directory(matrix_folder, matrix_list, ".mdf",test_sc)
    operating_exp = str(op_cost) + "*" + matrix_list[sov_dist_am] + "*("

    for mode in auto_sov_dist_modes:
        operating_exp += matrix_list[mode] + "+"
    operating_exp = operating_exp[:-1] + ")"

    operating_exp += "+" + str(op_cost) + "*" + matrix_list[sovt_dist_am] + "*("
    for mode in auto_sovt_dist_modes:
        operating_exp += matrix_list[mode] + "+"
    operating_exp = operating_exp[:-1] + ")"

    operating_exp += "+" + str(op_cost) + "*" + matrix_list[hov2_dist_am]+ "*" + matrix_list[trip_am_auto2nth]
    operating_exp += "+" + str(op_cost) + "*" + matrix_list[hov2t_dist_am] + "*" + matrix_list[trip_am_auto2th]
    operating_exp += "+" + str(op_cost) + "*" + matrix_list[hov3_dist_am] + "*" + matrix_list[trip_am_auto3nth]
    operating_exp += "+" + str(op_cost) + "*" + matrix_list[hov3t_dist_am]+ "*" + matrix_list[trip_am_auto3th]
    operating_exp = "(" + operating_exp + ")"

    parking_exp = parking_d + "*" + str(parking_factor) +  "*("
    for mode in all_auto:
        parking_exp += matrix_list[mode] + "+"
    parking_exp = parking_exp[:-1] + ")"

    toll_exp = matrix_list[hov2_toll_am] + "/100*(" 
    for mode in toll_hov2:
        toll_exp += matrix_list[mode] + "+"
    toll_exp = toll_exp[:-1] + ") + "+ matrix_list[hov3_toll_am] + "/100*(" 
    for mode in toll_hov3:
        toll_exp += matrix_list[mode] + "+"
    toll_exp = toll_exp[:-1] + ")"
    
    auto_cost_exp = "(" + operating_exp +"+" + parking_exp + "+" + toll_exp + ")"
    
    go_rail_fare_mat = eb.available_matrix_identifier('FULL')

    create_matrix(matrix_id = go_rail_fare_mat,
                      matrix_name= "go_rail_fare",
                      matrix_description = "GO Rail fare based on distance")

    go_rail_calc = deepcopy(mcSpec)
    go_rail_calc["expression"] = str(go_base_fare) + "+" + str(go_dist_fare) + "*" + matrix_list[pk_train_dist]
    go_rail_calc["constraint"]["by_value"] = {
                "interval_min": 0,
                "interval_max": 0,
                "condition": "EXCLUDE",
                "od_values":matrix_list[pk_train_dist]}
    go_rail_calc["result"] = go_rail_fare_mat

    matrix_calculator(go_rail_calc, scenario = test_sc)

    gorail_cost_exp = matrix_list[trips_am_train] + "*" + go_rail_fare_mat + "*" + str(go_rail_phf)
    subway_cost_exp = matrix_list[trips_am_subway] + "*" + str(subway_fare) + "*" + str(subway_phf)
    local_bus_cost_exp = matrix_list[trips_am_localbus] + "*" + local_fare_mat + "*" + str(local_bus_phf)
    go_bus_cost_exp = matrix_list[trips_am_expressbus] + "*" + matrix_list[pk_gobus_fare] + "*" + str(go_bus_phf)
    rapid_bus_cost_exp = matrix_list[trips_am_rapidbus] + "*" + local_fare_mat + "*" + str(rapid_bus_phf)
    ztsa_cost_exp = matrix_list[trips_am_ztsa] + "*" + matrix_list[pk_ztsa_fare] + "*" + str(local_bus_phf)

    transit_cost_exp = "(" + gorail_cost_exp +"+" + subway_cost_exp  +"+" + local_bus_cost_exp  +"+" + \
    go_bus_cost_exp  +"+" + rapid_bus_cost_exp  +"+" + ztsa_cost_exp + ")"
    
    moef.write(scen + ",Average Trip Cost,030104,")
    for origins in zones_array_st:
        spec = get_mc_spec(None, auto_cost_exp,"+","+",origins,"all",None)
        cost = matrix_calculator(spec, scenario = test_sc)['result']
        spec = get_mc_spec(None, transit_cost_exp,"+","+",origins,"all",None)
        transit_cost = matrix_calculator(spec, scenario = test_sc)['result']
        spec = get_mc_spec(None, pop_o_matrix,"+",None,origins,None,None)
        pop = matrix_calculator(spec, scenario = test_sc)['result']

        trip_cost = (cost + transit_cost)/pop
        moef.write("{0:.4f},".format(trip_cost))
    moef.write("\n")
    
    moef.write(scen + ",Average Trip Cost for residents in predominantly low-income zones,030109,")
    for origins in zones_array_st:
        spec = get_mc_spec(None, "(" +auto_cost_exp + ")" + "*" + low_income_zones_stations,"+","+",origins,"all",None)
        cost = matrix_calculator(spec, scenario = test_sc)['result']
        spec = get_mc_spec(None, "(" +transit_cost_exp+ ")" + "*" + low_income_zones_stations,"+","+",origins,"all",None)
        transit_cost = matrix_calculator(spec, scenario = test_sc)['result']
        spec = get_mc_spec(None, pop_o_matrix+ "*" + low_income_zones_stations,"+",None,origins,None,None)
        pop = matrix_calculator(spec, scenario = test_sc)['result']

        trip_cost = (cost + transit_cost)/pop
        moef.write("{0:.4f},".format(trip_cost))
    moef.write("\n")
moef.close()
print("Finished Processing {}".format(scen))

Now processing 2051_N2_Impro1
partition updated
skim.peak.transit.go_bus.fares
skim.am_peak.auto.hov2t.dist
skim.peak.transit.ztsa.fares
trips_am_peak_auto2_nt_hov
skim.am_peak.auto.hov3.dist
trips_peak_transit_express_bus
skim.am_peak.auto.hov3t.toll_cost
skim.am_peak.auto.hov2.dist
skim.peak.transit.train.distance
skim.am_peak.auto.hov3t.dist
trips_peak_transit_ztsa
trips_am_peak_auto1_nt
skim.am_peak.auto.hov2t.toll_cost
skim.peak.transit.subway.distance
trips_am_peak_auto2_nt_nh
skim.am_peak.auto.sov.dist
trips_am_peak_auto3_toll_hov
trips_am_peak_auto3_nt_hov
skim.am_peak.auto.sovt.toll_cost
trips_am_peak_auto2_toll_nh
trips_peak_transit_subway
trips_peak_transit_rapid_bus
trips_peak_transit_train
trips_peak_transit_local_bus
skim.am_peak.auto.sovt.dist
trips_am_peak_auto3_nt_nh
trips_am_peak_auto2_toll_hov
trips_am_peak_auto3_toll_nh
trips_am_peak_auto1_toll
Imported matrix mf8
Imported matrix mf2
Imported matrix mf13
Imported matrix mf5
Imported matrix mf10
Imported matrix mf7
I

In [24]:
moef.close()

In [25]:
out_file = r"C:\Users\PECHEN\Documents\Preferred Network\20210329_BAU_AvgCostBreakdown.csv"

moef = open(out_file, 'w')
moef.write("MOE," + ",".join(zones_array) + "\n")
#Connected Goal MOEs
#Objective 1

moef.write("Parking Cost,")
for origins in zones_array_st:
    spec = get_mc_spec(None, parking_exp,"+","+",origins,"all",None)
    result = matrix_calculator(spec)['result']
    moef.write("{0:.4f},".format(result))
print("Sub-element calculated")
moef.write("\n")

moef.write("Operating Cost,")
for origins in zones_array_st:
    spec = get_mc_spec(None, operating_exp,"+","+",origins,"all",None)
    result = matrix_calculator(spec)['result']
    moef.write("{0:.4f},".format(result))
print("Sub-element calculated")
moef.write("\n")

moef.write("Toll Cost,")
for origins in zones_array_st:
    spec = get_mc_spec(None, toll_exp,"+","+",origins,"all",None)
    result = matrix_calculator(spec)['result']
    moef.write("{0:.4f},".format(result))
print("Sub-element calculated")
moef.write("\n")

moef.write("GO Rail Cost,")
for origins in zones_array_st:
    spec = get_mc_spec(None, gorail_cost_exp,"+","+",origins,"all",None)
    result = matrix_calculator(spec)['result']
    moef.write("{0:.4f},".format(result))
print("Sub-element calculated")
moef.write("\n")

moef.write("Subway Cost,")
for origins in zones_array_st:
    spec = get_mc_spec(None, subway_cost_exp,"+","+",origins,"all",None)
    result = matrix_calculator(spec)['result']
    moef.write("{0:.4f},".format(result))
print("Sub-element calculated")
moef.write("\n")

moef.write("Rapid Bus Cost,")
for origins in zones_array_st:
    spec = get_mc_spec(None, rapid_bus_cost_exp,"+","+",origins,"all",None)
    result = matrix_calculator(spec)['result']
    moef.write("{0:.4f},".format(result))
print("Sub-element calculated")
moef.write("\n")

moef.write("Express Bus Cost,")
for origins in zones_array_st:
    spec = get_mc_spec(None, go_bus_cost_exp,"+","+",origins,"all",None)
    result = matrix_calculator(spec)['result']
    moef.write("{0:.4f},".format(result))
print("Sub-element calculated")
moef.write("\n")

moef.write("Local Bus Cost,")
for origins in zones_array_st:
    spec = get_mc_spec(None, local_bus_cost_exp,"+","+",origins,"all",None)
    result = matrix_calculator(spec)['result']
    moef.write("{0:.4f},".format(result))
print("Sub-element calculated")
moef.write("\n")

moef.write("ZTSA Cost,")
for origins in zones_array_st:
    spec = get_mc_spec(None, ztsa_cost_exp,"+","+",origins,"all",None)
    result = matrix_calculator(spec)['result']
    moef.write("{0:.4f},".format(result))
print("Sub-element calculated")
moef.write("\n")

moef.write("Population,")
for origins in zones_array:
    spec = get_mc_spec(None, pop_o_matrix,"+",None,origins,None,None)
    pop = matrix_calculator(spec)['result']
    moef.write("{0:.4f},".format(pop))
moef.write("\n")

moef.close()

Sub-element calculated
Sub-element calculated
Sub-element calculated
Sub-element calculated
Sub-element calculated
Sub-element calculated
Sub-element calculated
Sub-element calculated
Sub-element calculated
